In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-06"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10507,2024-05-06,Rússia Liga Vtb United,13:00,Unics Kazan,Lokomotiv Kuban,1.45,2.77,164.5,1.83,1.93,0.0,0.00,0.00,zRdNXP4s,0.689655,0.361011,0.546448,0.518135,0.050666,129.816,47.785712,0.368103,2.4,1.341641,0.559017,122.40,1.514,0.511596,0.337910,52.0,124.736,20.417499,0.163686,1.8,1.643168,0.912871,123.82,1.388,0.220273,0.158698,80.0,90,82,1.36,1.51,115.368,166.714,0.442361,0.037612,NaN,0.45,0.090,5.000000,0.774582,0.8,0.025418,-0.30,-0.060,-29.500000,0.559982,0.5,-0.059982
10508,2024-05-06,República Tcheca Nbl,14:00,Nymburk,Ostrava,1.02,11.10,158.5,1.83,1.87,0.0,0.00,0.00,tMIiNGse,0.980392,0.090090,0.546448,0.534759,0.070482,96.020,16.773139,0.174684,3.0,0.000000,0.000000,94.76,1.042,0.026833,0.025751,105.0,359.208,211.271146,0.588158,1.8,1.643168,0.912871,683.10,4.714,3.217558,0.682554,-28.0,92,69,1.03,9.90,91.896,391.826,1.176178,0.015289,NaN,0.14,0.028,0.714286,0.946525,1.0,0.053475,2.24,0.448,22.544643,0.395715,0.5,0.104285
10509,2024-05-06,Lituânia Lkl,12:50,Lietkabelis,Nevezis,1.19,4.37,172.5,1.86,1.84,0.0,0.00,0.00,8StSOymd,0.840336,0.228833,0.537634,0.543478,0.069169,152.816,82.642741,0.540799,1.2,1.643168,1.369306,104.72,1.700,0.762430,0.448488,3.0,442.388,291.988574,0.660028,0.6,1.341641,2.236068,904.80,5.328,3.954544,0.742219,-27.0,88,78,1.19,11.60,144.168,473.418,0.808849,0.007644,NaN,-2.72,-0.544,-0.349265,0.701461,0.5,-0.201461,-1.54,-0.308,-10.941558,0.295007,0.2,-0.095007
10510,2024-05-06,Paraguai Lnb,20:30,Felix Perez Cardozo,Dep. San Jose,8.55,1.05,157.5,1.81,1.85,0.0,0.00,0.00,foKxcwS1,0.116959,0.952381,0.552486,0.540541,0.069340,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,70,89,4.47,1.37,0.000,0.000,1.104854,0.015456,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10511,2024-05-06,Paraguai Lnb,20:30,Ciudad Nueva,Sol De America,1.09,6.52,152.5,1.87,1.79,0.0,0.00,0.00,bJQTback,0.917431,0.153374,0.534759,0.558659,0.070805,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,62,0,13.00,0.00,0.000,0.000,1.009091,0.030912,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10512,2024-05-06,Rússia Superliga,10:00,Temp Sumz Revda,Zenit Petersburg 2,1.10,5.39,148.5,1.83,1.83,0.0,0.00,0.00,UiGU2ImR,0.909091,0.185529,0.546448,0.546448,0.094620,118.140,31.689632,0.268238,2.4,1.341641,0.559017,157.76,1.732,0.680786,0.393064,23.0,339.592,112.111409,0.330136,1.8,1.643168,0.912871,478.71,4.262,1.403325,0.329264,-20.0,58,81,2.72,5.91,88.796,335.002,0.934819,0.000000,NaN,-0.47,-0.094,-1.063830,0.000000,0.0,0.000000,5.36,1.072,4.095149,0.000000,0.0,0.000000
10513,2024-05-06,Paraguai Lnb,20:30,Libertad,Olimpia,4.42,1.18,155.5,1.85,1.85,0.0,0.00,0.00,hKbdH7DG,0.226244,0.847458,0.540541,0.540541,0.073702,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,59,85,6.57,1.18,0.000,0.000,0.818224,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10514,2024-05-06,Taiwan Liga T1,08:00,New Taipei CTBC DEA,Taipei Tais

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10544,14:15,Israel Superliga,Ironi Kiryat Ata,Hapoel Jerusalem,2.49,Back Home
10545,14:10,Israel Superliga,H. Haifa,Hapoel Beer Sheva,1.61,Back Home
10548,16:00,Itália Série A2,Sebastiani Rieti,Rimini,1.61,Back Home
